In [1]:
#imports
import os
from owlready2 import get_ontology
from owlready2 import sync_reasoner_hermit
import traceback

In [2]:
# Define the file path
directory = 'C:\\Users\\Matheus\\Documents\\GitHub\\Papers\\Enabling Intelligent Data Exchange in the Brazilian Energy Sector A Context-Aware Ontological Approach\\ontology\\imports'
filename = 'oec-extracted.owl'
file_path = os.path.join(directory, filename)

try:
    # Load the ontology
    onto = get_ontology("file://" + file_path).load()
    print("Ontology loaded successfully!")

except Exception as e:
    # Print an error message if loading fails
    print("Error occurred during ontology loading:")
    print(e)

#check inconsistensies
inconsistencies = list(onto.inconsistent_classes())
if inconsistencies:
    print("Ontology is inconsistent!")
    print("Inconsistencies:")
    for inconsistency in inconsistencies:
        print(f"Inconsistency found involving: {inconsistency}")
else:
    print("Ontology is consistent.")    

Ontology loaded successfully!
Ontology is consistent.


In [3]:
#sync reasoner
try:
    with onto:
        sync_reasoner_hermit()
except Exception as e:
    print(f"Error: {e}")


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp c:\ProgramData\anaconda3\Lib\site-packages\owlready2\hermit;c:\ProgramData\anaconda3\Lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/Matheus/AppData/Local/Temp/tmpmyg3ts_l
* Owlready2 * HermiT took 0.5959796905517578 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [ ]:
# Get inferred object properties and their usage
inferred_properties = set()

for prop in onto.object_properties():
    try:
        for subj, obj in prop.get_relations():
            inferred_properties.add((prop, subj, obj))
    except Exception as e:
        print(f"An error occurred while processing object property '{prop.name}': {e}")

# Print inferred object properties and their usage
for prop, subj, obj in inferred_properties:
    try:
        subj_type = "Class" if subj in onto.classes() else "Individual" if subj in onto.individuals() else "Unknown"
        obj_type = "Class" if obj in onto.classes() else "Individual" if obj in onto.individuals() else "Unknown"

        print(f"Object Property '{prop.name}' inferred on {obj_type.lower()} '{obj.name}' because of '{subj.name}'")
    except Exception as e:
        print(f"An error occurred while formatting output: {e}")

In [ ]:
# Define the SPARQL query
query = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX oec: <http://www.semanticweb.org/matheus/ontologies/2023/10/oec-extracted#>

    SELECT ?individual1 ?individual2
    WHERE {
        ?individual1 oec:termHasMeaningByContext ?individual2 .
    }
"""

try:
    # Execute SPARQL query
    results = onto.world.sparql(query)

    # Print results
    for row in results:
        individual1 = row[0]
        individual2 = row[1]
        print(f"Individual connected through 'termHasMeaningByContext': {individual1} to {individual2}")

except Exception as e:
    print("Error occurred while executing the SPARQL query:")
    print(e)

In [ ]:
# Print inferred information for each class, property, and individual
print("Inferred Classes:")
for cls in onto.classes():
    cls_name = cls.name.split('#')[-1] if '#' in cls.name else cls.name
    print(f"\nClass: {cls_name}")
    instances = list(cls.instances())
    if instances:
        instance_names = [inst.name.split('#')[-1] if '#' in inst.name else inst.name for inst in instances]
        print(f"Inferred Instances: {instance_names}")
    else:
        print("No inferred instances.")

print("\nInferred Object Properties:")
for prop in onto.object_properties():
    prop_name = prop.name.split('#')[-1] if '#' in prop.name else prop.name
    print(f"\nObject Property: {prop_name}")
    relations = [(subj.name.split('#')[-1] if '#' in subj.name else subj.name,
                  obj.name.split('#')[-1] if '#' in obj.name else obj.name) for subj, obj in prop.get_relations()]
    if relations:
        for subj, obj in relations:
            print(f"Subject: {subj}, Object: {obj}")
    else:
        print("No inferred relations.")

print("\nInferred Data Properties:")
for data_prop in onto.data_properties():
    data_prop_name = data_prop.name.split('#')[-1] if '#' in data_prop.name else data_prop.name
    print(f"\nData Property: {data_prop_name}")
    values = [(subj.name.split('#')[-1] if '#' in subj.name else subj.name, value) for subj, value in data_prop.get_relations()]
    if values:
        for subj, value in values:
            print(f"Subject: {subj}, Value: {value}")
    else:
        print("No inferred values.")

print("\nInferred Individuals:")
for ind in onto.individuals():
    ind_name = ind.name.split('#')[-1] if '#' in ind.name else ind.name
    print(f"\nIndividual: {ind_name}")
    types = [typ.name.split('#')[-1] if '#' in typ.name else typ.name for typ in ind.is_a]
    if types:
        print(f"Inferred Types: {types}")
    else:
        print("No inferred types.")